<a href="https://colab.research.google.com/github/fstigler/AI_Programming_with_Python_Udacity_Nanodegree-/blob/main/stock_strategy_S%26P500.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

     |████████████████████████████████| 6.3 MB 10.1 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.64-py2.py3-none-any.whl size=24109 sha256=11413bdb33fdca94b596d74742538045376f3f04e1f43f708162ccb2bdffe8af
  Stored in directory: /root/.cache/pip/wheels/86/fe/9b/a4d3d78796b699e37065e5b6c27b75cff448ddb8b24943c288
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
!pip install pandas

In [ ]:
!pip install numpy

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

In [ ]:
df = yf.download('PLTR')

[*********************100%***********************]  1 of 1 completed


In [ ]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-09-30,10.000000,11.410000,9.110000,9.500000,9.500000,338584400
2020-10-01,9.690000,10.100000,9.230000,9.460000,9.460000,124297600
2020-10-02,9.060000,9.280000,8.940000,9.200000,9.200000,55018300
2020-10-05,9.430000,9.490000,8.920000,9.030000,9.030000,36316900
2020-10-06,9.040000,10.180000,8.900000,9.900000,9.900000,90864000
...,...,...,...,...,...,...
2021-10-19,24.500000,25.200001,24.360001,24.690001,24.690001,29393900
2021-10-20,24.790001,24.959999,24.170000,24.219999,24.219999,24248000
2021-10-21,24.129999,24.930000,24.080000,24.780001,24.780001,23976600


In [ ]:
df['SMA200'] = df.Close.rolling(100).mean()

In [ ]:
df['SMA5'] = df.Close.rolling(5).mean()

In [ ]:
df['10d_low'] = df.Close.rolling(10).min()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df['Buy'] = np.where((df.Close > df.SMA200) & 
                     (df['10d_low'].diff() < 0) &
                    (0.98 * df.Close >= df.Low.shift(-1)), 1, 0)

In [ ]:
df['Sell'] = np.where(df.Close > df.SMA5, 1, 0)

In [ ]:
df['Buyprice'] = 0.98 * df.Close

In [ ]:
df['Sellprice'] = df.Open.shift(-1)

In [ ]:
x = df[(df.Buy ==1) | (df.Sell ==1)]

In [ ]:
y = x[(x.Buy.diff() ==1) | (x.Sell.diff() ==1)]

In [ ]:
profits = ((y.Sellprice.shift(-1) - y.Buyprice)/y.Buyprice)

In [ ]:
profits = profits[::2]

In [ ]:
def backtest(arr):
  winrate = len(arr[arr > 0])/len(arr)
  max_dd = min(arr)
  mean = arr.mean()
  gain = (arr +1).cumprod()[-1]
  return winrate, max_dd, mean, gain

In [ ]:
backtest(profits)

(0.5, 0.056718728983406866, 0.056718728983406866, nan)

In [ ]:
df.tail(10)

,Open,High,Low,Close,Adj Close,Volume,SMA200,SMA5,10d_low,Buy,Sell,Buyprice,Sellprice
Date,,,,,,,,,,,,,
2021-10-12,23.549999,23.809999,23.290001,23.690001,23.690001,26280000,24.3372,23.606,23.170000,0,1,23.216201,23.830000
2021-10-13,23.830000,24.309999,23.790001,24.129999,24.129999,28494000,24.3710,23.716,23.170000,0,1,23.647399,24.430000
2021-10-14,24.430000,24.709999,24.205000,24.350000,24.350000,26106700,24.4023,23.840,23.170000,0,1,23.863000,24.500000
2021-10-15,24.500000,24.520000,23.830000,24.000000,24.000000,24275000,24.4275,23.940,23.170000,0,1,23.520000,24.000000
2021-10-18,24.000000,24.389999,23.920000,24.260000,24.260000,25609900,24.4489,24.086,23.209999,0,1,23.774800,24.500000
2021-10-19,24.500000,25.200001,24.360001,24.690001,24.690001,29393900,24.4688,24.286,23.500000,0,1,24.196201,24.790001
2021-10-20,24.790001,24.959999,24.170000,24.219999,24.219999,24248000,24.4815,24.304,23.500000,0,0,23.735599,24.129999
2021-10-21,24.129999,24.930000,24.080000,24.780001,24.780001,23976600,24.4987,24.390,23.500000,0,1,24.284401,24.490000
2021-10-22,24.490000,24.850000,24.250000,24.430000,24.430000,20098800,24.4985,24.476,23.530001,0,0,23.941400,24.500000


In [ ]:
def manipulatedf(df):
  df = df.copy()
  df['SMA200'] = df.Close.rolling(200).mean()
  df['SMA5'] = df.Close.rolling(5).mean()
  df['10d_low'] = df.Close.rolling(10).min()
  df.dropna(inplace=True)
  df['Buy'] = np.where((df.Close > df.SMA200) & 
                     (df['10d_low'].diff() < 0) &
                    (0.98 * df.Close >= df.Low.shift(-1)), 1, 0)
  df['Sell'] = np.where(df.Close > df.SMA5, 1, 0)
  df['Buyprice'] = 0.98 * df.Close
  df['Sellprice'] = df.Open.shift(-1)
  x = df[(df.Buy ==1) | (df.Sell ==1)]
  y = x[(x.Buy.diff() ==1) | (x.Sell.diff() ==1)]
  profits = ((y.Sellprice.shift(-1) - y.Buyprice)/y.Buyprice)
  profits = profits[::2]
  return profits

In [ ]:
backtest(manipulatedf(df))

(1.0, 0.0013213287397033002, 0.020457519660100273, 1.4960189155514463)

In [ ]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

In [ ]:
tickers = tickers.Symbol.to_list()


In [ ]:
tickers

['MMM',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AMD',
 'AAP',
 'AES',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'AOS',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BLL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'CHRW',
 'CDNS',
 'CZR',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CDAY',
 'CERN',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMC

In [ ]:
profitmatrix = []

for ticker in tickers:
  df = yf.download(ticker, start = '2000-01-01')
  prof = manipulatedf(df)
  profitmatrix.append(prof)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
profitmatrix = [i for i in profitmatrix if len(i) >0]

In [ ]:
results = []

for ele in profitmatrix: 
  backtest(ele)
  results.append(backtest(ele))

In [ ]:
frame = pd.DataFrame(results)

In [ ]:
frame.columns = ['winrate', 'max_dd', 'mean', 'gain']

In [ ]:
frame.mean()

winrate    0.668237
max_dd    -0.154725
mean       0.010186
gain       1.433301
dtype: float64